In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [28]:
df_train = pd.read_csv('data/train.csv', header=None, sep='\,')
df_test = pd.read_csv('data/test.csv', header=None, sep='\,')

C:\Users\wens_120\anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Users\wens_120\anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [61]:
X_train = df_train.iloc[:, :-4]
y_train = df_train.iloc[:, -3]
X_test = df_test.iloc[:, :-4]
y_test = df_test.iloc[:, -3]

In [78]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,id,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
1,0,25,0.3795,0.42993,0.52076,0.57166,0.67818,0.75476,0.8358,0.93623,...,NaN,0.0,NaN,1.0675043174963164e-18,5.998948958141868e-18,4.3785130733854596e-17,5.59,4.32,8.92,4.29
2,1,10,0.0,0.0,0.01813,0.0,0.0,0.01974,0.00321,0.0,...,1.3431318575117464e-08,6.11268485495327e-09,2.130546870046817e-09,NaN,9.710091008671586e-09,NaN,0.0,2.83,7.25,4.64
3,2,25,0.0,0.03289,0.02416,0.0361,0.05843,0.09015,0.14944,0.18578,...,0.0,0.0,0.0,0.0,1.329724810420566e-18,NaN,10.64,3.0,8.4,5.16
4,3,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,...,2.2459980118957924e-10,1.2995108396165954e-10,7.782625483970989e-11,NaN,4.0889209621275374e-10,NaN,5.67,4.01,5.05,4.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,9995,15,0.23929,0.30265,0.39929,0.51,0.64072,0.77328,0.86722,0.95891,...,6.788642458257553e-16,0.0,5.516466876499084e-16,9.690978764713718e-16,1.3916348723977759e-15,5.460701991896471e-14,12.68,4.11,12.31,0.1
9997,9996,20,0.02583,0.00946,0.0365,0.0138,0.04093,0.04363,0.0326,0.05282,...,1.8142744089936586e-16,1.1171478479246386e-16,7.352416002936492e-17,NaN,5.825346509976335e-16,4.226435934781879e-15,8.46,4.11,10.46,3.12
9998,9997,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,...,4.8284512492038765e-11,3.11041865784904e-11,8.119094936563449e-12,NaN,1.5041154520016192e-10,4.0036568364409913e-10,9.84,3.2,10.45,2.06
9999,9998,15,1.01477,1.01504,0.99125,0.98747,1.00717,1.01434,0.99529,1.01322,...,3.2792711729643687e-13,0.0,4.4759455225507956e-14,NaN,2.8188565909283684e-12,1.4087424191873974e-11,6.38,4.06,11.28,4.03


In [104]:
y_train = y_train[0]
y_test = y_test[0]
X_train = X_train.drop(0, axis=1)
X_test = X_test.drop(0, axis=1)

SyntaxError: invalid syntax (<ipython-input-104-ebbb3bb1d9f8>, line 3)

# 결측치 제거

In [80]:
X_train = X_train.fillna(0)
# y_train = y_train.fillna(0)
X_test = X_test.fillna(0)
# y_test = y_test.fillna(0)

In [103]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,id,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
1,0,25,0.3795,0.42993,0.52076,0.57166,0.67818,0.75476,0.8358,0.93623,...,NaN,0.0,NaN,1.0675043174963164e-18,5.998948958141868e-18,4.3785130733854596e-17,5.59,4.32,8.92,4.29
2,1,10,0.0,0.0,0.01813,0.0,0.0,0.01974,0.00321,0.0,...,1.3431318575117464e-08,6.11268485495327e-09,2.130546870046817e-09,NaN,9.710091008671586e-09,NaN,0.0,2.83,7.25,4.64
3,2,25,0.0,0.03289,0.02416,0.0361,0.05843,0.09015,0.14944,0.18578,...,0.0,0.0,0.0,0.0,1.329724810420566e-18,NaN,10.64,3.0,8.4,5.16
4,3,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,...,2.2459980118957924e-10,1.2995108396165954e-10,7.782625483970989e-11,NaN,4.0889209621275374e-10,NaN,5.67,4.01,5.05,4.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,9995,15,0.23929,0.30265,0.39929,0.51,0.64072,0.77328,0.86722,0.95891,...,6.788642458257553e-16,0.0,5.516466876499084e-16,9.690978764713718e-16,1.3916348723977759e-15,5.460701991896471e-14,12.68,4.11,12.31,0.1
9997,9996,20,0.02583,0.00946,0.0365,0.0138,0.04093,0.04363,0.0326,0.05282,...,1.8142744089936586e-16,1.1171478479246386e-16,7.352416002936492e-17,NaN,5.825346509976335e-16,4.226435934781879e-15,8.46,4.11,10.46,3.12
9998,9997,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,...,4.8284512492038765e-11,3.11041865784904e-11,8.119094936563449e-12,NaN,1.5041154520016192e-10,4.0036568364409913e-10,9.84,3.2,10.45,2.06
9999,9998,15,1.01477,1.01504,0.99125,0.98747,1.00717,1.01434,0.99529,1.01322,...,3.2792711729643687e-13,0.0,4.4759455225507956e-14,NaN,2.8188565909283684e-12,1.4087424191873974e-11,6.38,4.06,11.28,4.03


In [98]:
def split_y(data, col) :
    X = data.iloc[:, :-4]
    y = data.iloc[:, -4:]
    
    y = y[col]
    
    data = pd.concat([X,y], axis=1)
    
    return data

In [102]:

train_hbo2 = split_y(df_train, 'hbo2')


KeyError: 'hbo2'

In [99]:
train_hbo2.drop(['id'], axis=1, inplace=True)
X = train_hbo2.iloc[:, :-1]
y = train_hbo2.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

NameError: name 'train_hbo2' is not defined

In [81]:
X_train

,1,2,3,4,5,6,7,8,9,10,...,62,63,64,65,66,67,68,69,70,71
0,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,900_dst,910_dst,920_dst,930_dst,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst
1,25,0.3795,0.42993,0.52076,0.57166,0.67818,0.75476,0.8358,0.93623,0.96333,...,0,3.527371055958131e-18,0,6.45556356054237e-19,0,0.0,0,1.0675043174963164e-18,5.998948958141868e-18,4.3785130733854596e-17
2,10,0.0,0.0,0.01813,0.0,0.0,0.01974,0.00321,0.0,0.0,...,2.6476332408267652e-09,0,5.233480332475007e-09,1.2642375538442756e-08,1.3431318575117464e-08,6.11268485495327e-09,2.130546870046817e-09,0,9.710091008671586e-09,0
3,25,0.0,0.03289,0.02416,0.0361,0.05843,0.09015,0.14944,0.18578,0.25584,...,4.629124827699046e-18,1.4094133246123881e-18,3.2374801151566597e-18,0.0,0.0,0.0,0.0,0.0,1.329724810420566e-18,0
4,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,0.79865,...,1.2748760176895748e-10,6.118946541281467e-11,0,1.6638877271624956e-10,2.2459980118957924e-10,1.2995108396165954e-10,7.782625483970989e-11,0,4.0889209621275374e-10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,15,0.23929,0.30265,0.39929,0.51,0.64072,0.77328,0.86722,0.95891,0.98998,...,0.0,2.314356831163149e-15,2.69414194493479e-15,0,6.788642458257553e-16,0.0,5.516466876499084e-16,9.690978764713718e-16,1.3916348723977759e-15,5.460701991896471e-14
9997,20,0.02583,0.00946,0.0365,0.0138,0.04093,0.04363,0.0326,0.05282,0.04608,...,2.0117693820348545e-16,0,1.6448591513549337e-17,6.861706334591001e-17,1.8142744089936586e-16,1.1171478479246386e-16,7.352416002936492e-17,0,5.825346509976335e-16,4.226435934781879e-15
9998,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,1.00493,...,6.89806777917326e-11,0,0,6.473638316062647e-11,4.8284512492038765e-11,3.11041865784904e-11,8.119094936563449e-12,0,1.5041154520016192e-10,4.0036568364409913e-10
9999,15,1.01477,1.01504,0.99125,0.98747,1.00717,1.01434,0.99529,1.01322,1.00486,...,0,3.149952984752282e-13,4.371710553245746e-14,0,3.2792711729643687e-13,0.0,4.4759455225507956e-14,0,2.8188565909283684e-12,1.4087424191873974e-11


In [96]:
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=1)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71

In [17]:
X_train

""
0
1
2
3
4
...
9996
9997
9998
9999
